In [4]:
import pandas as pd
import re

review_df = pd.read_csv('C:\\Users\\HANA\\PycharmProjects\\HANATOUR\\NLP\\TEXT_Example\\labeledTrainData.tsv', header=0, sep="\t", quoting=3)

print(review_df.head())
# print(review_df['review'][0])

#불필요한 내용 정리 하기에서 위와 아래는 동일한 내용
review_df['review'] = review_df['review'].str.replace('<br />', ' ')
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]"," ", x))

         id  sentiment                                             review
0  "5814_8"          1  "With all this stuff going down at the moment ...
1  "2381_9"          1  "\"The Classic War of the Worlds\" by Timothy ...
2  "7759_3"          0  "The film starts with a manager (Nicholas Bell...
3  "3630_4"          0  "It must be assumed that those who praised thi...
4  "9495_8"          1  "Superbly trashy and wondrously unpretentious ...


In [6]:
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
print(class_df)

feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)
print(feature_df)

X_train, X_test, y_train, y_test = train_test_split(feature_df, class_df, test_size=0.3, random_state=156)
print(X_train.shape, X_test.shape)

0        1
1        1
2        0
3        0
4        1
        ..
24995    0
24996    0
24997    0
24998    0
24999    1
Name: sentiment, Length: 25000, dtype: int64
                                                  review
0       With all this stuff going down at the moment ...
1         The Classic War of the Worlds   by Timothy ...
2       The film starts with a manager  Nicholas Bell...
3       It must be assumed that those who praised thi...
4       Superbly trashy and wondrously unpretentious ...
...                                                  ...
24995   It seems like more consideration has gone int...
24996   I don t believe they made this film  Complete...
24997   Guy is a loser  Can t get girls  needs to bui...
24998   This    minute documentary Bu uel made in the...
24999   I saw this movie as a child and it broke my h...

[25000 rows x 1 columns]
(17500, 1) (7500, 1)


In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

pipline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lt_clf', LogisticRegression(C=10))])

In [8]:
#Pipline 객체를 이용해 fit(), prefit()로 학습/예측 수행, predict_proba()는 roc_auc때문에 수행
pipline.fit(X_train['review'], y_train)
pred = pipline.predict(X_test['review'])
pred_prods = pipline.predict_proba(X_test['review'])[:, 1]
print(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_prods))

c:\users\hana\pycharmprojects\hanatour\venv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.886 0.9502703875483725


In [9]:
#스톱 워드는 English, Filtering, ngram은 (1,2)로 설정해 TfidfVectorizer 수행
#LogisticRegression의 C는 10으로 설정
pipline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2))),
    ('lt_clf', LogisticRegression(C=10))])

pipline.fit(X_train['review'], y_train)
pred = pipline.predict(X_test['review'])
pred_prods = pipline.predict_proba(X_test['review'])[:, 1]
print(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_prods))

0.8936 0.959799823582973
